# artist-genre-evolution
This notebook is submitted as a prerequisite of CSMODEL and uses the [Spotify Music Dataset](https://www.kaggle.com/datasets/solomonameh/spotify-music-dataset) from Kaggle.

Our main goal is to answer the question **"Do artists in certain music genres exhibit more change in their musical features over time than others?"**

ie: Does the genre an artist is in affect how much they *change* or *evolve* in terms of their musical features (danceabiltiy, energy, acousticness).



## About the Dataset
__________

This project utilizes the "High Popularity Songs" portion of the Spotify Music Dataset, sourced from Kaggle. This dataset is a curated collection of songs from Spotify that are considered popular, specifically those **with a popularity score greater than 68.** 

For each song, the dataset provides a rich set of attributes, including quantitative audio features derived from Spotify's audio analysis (e.g., `danceability`, `energy`, `valence`) and descriptive metadata (e.g., `track_name`, `track_artist`, `release_date`).

## Data Collection & Potential Implications


The dataset was collated by querying the official Spotify Web API using custom Python scripts. The data for each track, including its audio features, is what Spotify provides directly to developers.

The method of collection has several key implications for our analysis:
-  **Popularity Bias**: The dataset is intentionally filtered to include only songs with a popularity score above 68. This means our insights will be specific to commercially successful music on the platform and cannot be generalized to all music or to less popular/niche genres. The factors driving popularity on Spotify (e.g., playlist placement, marketing, platform promotion) are complex and may introduce a bias towards mainstream music.

While there is a low_popularity_songs dataset, the researchers agreed not to use it because the primary research question requires analyzing an artist's musical evolution over time. **High-popularity artists are more likely to have extensive discographies with multiple release dates, providing sufficient data for a longitudinal analysis.** Conversely, low-popularity artists often have too few songs in the dataset to establish a meaningful timeline of change

- **Standardized Metrics**: Since the audio features (energy, liveness, etc.) are calculated by Spotify's proprietary algorithms, they represent a consistent and standardized measurement system. However, these metrics are a "black box," and we must rely on Spotify's definitions for their interpretation.

- **Temporal Relevance**: The dataset represents a snapshot in time. Song popularity is highly dynamic, and the trends identified may be specific to the period in which the data was collected.

- **Playlist-Based Genre**: The `playlist_genre` and `playlist_subgenre` fields are derived from the playlists the songs appeared in, not from an official artist-level genre tag. An artist's songs may appear in playlists of various genres (e.g., a rock song in a "Workout" playlist categorized under Pop). This means our genre groupings are an approximation and could introduce noise into the analysis.


## Structure 


The data is presented in a tabular format within the `high_popularity_spotify_data.csv` file.
- Rows: Each row represents a single, unique song (track).
- Columns: Each column represents a specific attribute or feature of that song.
- Observations: The dataset contains 1686 observations (songs) and 58 features (columns).



## Attributes 



The 58 features can be broadly categorized into Audio Features and Descriptive Features.

**Audio Features**

These are quantitative features generated by Spotify's analysis of a track's audio.

- `energy`: A measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy.
- `tempo`: The speed of a track, measured in beats per minute (BPM).

- `danceability`: A score describing how suitable a track is for dancing based on tempo, rhythm stability, beat strength, and overall regularity.

- `loudness`: The overall loudness of a track in decibels (dB).

- `liveness`: The likelihood of a track being a live recording. Higher values suggest the presence of an audience.

- `valence`: The musical positiveness (emotion) of a track. High valence sounds happy; low valence sounds sad or angry.

- `speechiness`: Measures the presence of spoken words in a track.

- `instrumentalness`: The likelihood a track contains no vocals. Values near 1.0 suggest purely instrumental tracks.

- `mode`: Indicates the modality (major or minor) of the track.

- `key`: The musical key, represented as an integer (0 - 11) mapping to Pitch Class notation.

- `duration_ms`: The length of the track in milliseconds.

- `acousticness`: A confidence measure of whether a track is acoustic.

**Descriptive Features**

These are metadata attributes that describe the track and its context.

- `track_name`: The name of the song.

- `track_artist`: The artist(s) who performed the song.

- `track_album_name`: The album the song belongs to.

- `track_album_release_date`: The release date of the album.

- `track_popularity`: A score (0 - 100) calculated by Spotify based on the total number of streams and how recent they are.

- `playlist_name`: The name of the playlist the track was sourced from.

- `playlist_genre:` The main genre associated with the source playlist.

- `playlist_subgenre`: A more specific subgenre of the source playlist.

- `track_id`: A unique identifier for the track, assigned by Spotify.

- `track_album_id`: A unique identifier for the album.

- `playlist_id`: A unique identifier for the source playlist.

## Data Cleaning and preprocessing

Before proceeding with our analysis, it is crucial to clean and preprocess the raw data to ensure accuracy, consistency, and suitability for our research question. This section details the steps taken to prepare the `high_popularity_spotify_data.csv` dataset.


sample formula formatting:
$$p \pm ME$$
$$ME =  z^* \times \sqrt{\frac{p(1-p)}{n}}$$

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


high_df = pd.read_csv('high_popularity_spotify_data.csv')
high_df.head()

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,...,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id
0,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0000,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
1,0.507,104.978,0.747,pop,-10.171,0.117,0.438,Billie Eilish,4,0.0358,...,0.0608,7aJuG4TFXa2hmE4z1yxc3n,1,2,210373,0.2000,6dOtVTDdiauQNBQEDOtlAB,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
2,0.808,108.548,0.554,pop,-4.169,0.159,0.372,Gracie Abrams,4,0.0368,...,0.0000,0hBRqPYPXhr1RkTDG3n4Mk,1,1,166300,0.2140,7ne4VBA60CxGM75vw0EYad,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
3,0.910,112.966,0.670,pop,-4.070,0.304,0.786,Sabrina Carpenter,4,0.0634,...,0.0000,4B4Elma4nNDUyl6D5PvQkj,0,0,157280,0.0939,1d7Ptw3qYcfpdLNL5REhtJ,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
4,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0000,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M


In [9]:
high_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   energy                    1686 non-null   float64
 1   tempo                     1686 non-null   float64
 2   danceability              1686 non-null   float64
 3   playlist_genre            1686 non-null   object 
 4   loudness                  1686 non-null   float64
 5   liveness                  1686 non-null   float64
 6   valence                   1686 non-null   float64
 7   track_artist              1686 non-null   object 
 8   time_signature            1686 non-null   int64  
 9   speechiness               1686 non-null   float64
 10  track_popularity          1686 non-null   int64  
 11  track_href                1686 non-null   object 
 12  uri                       1686 non-null   object 
 13  track_album_name          1685 non-null   object 
 14  playlist

### Standardizing Categorical Text

To ensure consistency and prevent different representations of the same category, we convert string-based categorical columns like `playlist_genre` and `track_artist` to lowercase and remove leading/trailing whitespace.


In [10]:
# This makes sure that the 'playlist_genre' and 'track_artist' columns are in lowercase and that there are no duplicates.
high_df['playlist_genre'] = high_df['playlist_genre'].str.lower().str.strip()
high_df['track_artist'] = high_df['track_artist'].str.lower().str.strip()

### Handling Missing Values
We first inspect the dataset for any missing values.

In [11]:
# Check for any null values
high_df.isnull().sum()

energy                      0
tempo                       0
danceability                0
playlist_genre              0
loudness                    0
liveness                    0
valence                     0
track_artist                0
time_signature              0
speechiness                 0
track_popularity            0
track_href                  0
uri                         0
track_album_name            1
playlist_name               0
analysis_url                0
track_id                    0
track_name                  0
track_album_release_date    0
instrumentalness            0
track_album_id              0
mode                        0
key                         0
duration_ms                 0
acousticness                0
id                          0
playlist_subgenre           0
type                        0
playlist_id                 0
dtype: int64

The output above indicates one missing value in the track_album_name column. Given that this is a single instance out of 1686 observations, we will drop this row as its impact on the overall analysis will be minimal.

In [12]:
# Clear any row with missing values
high_df = high_df.dropna()

### Managing Duplicate Song Entries from Playlists

The dataset is compiled from various Spotify playlists. This can lead to the same `track_id` (representing a unique song) appearing multiple times if it's featured in different playlists. 

While not true duplicates in terms of unique (`track_id`, `playlist_id`) pairs, this can introduce complexities when analyzing unique song properties or artist-genre relationships if a song is associated with multiple genres.

We first check for exact duplicates across `playlist_genre`, `id` (track_id), and `playlist_subgenre`:

In [13]:
# Check for any duplicate rows that may interfere with the EDA
high_df.duplicated(subset=['playlist_genre', 'id', 'playlist_subgenre']).any()

np.False_

This indicates that no song appears in the exact same playlist genre and subgenre combination more than once. However, a song can appear in different playlist genres or subgenres. Let's inspect where `track_id` and `playlist_genre` are duplicated:

In [14]:
high_df_dups = high_df[high_df.duplicated(subset=['playlist_genre', 'id'])]
high_df_dups

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,...,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id
180,0.472,101.061,0.898,hip-hop,-7.001,0.141,0.2140,kendrick lamar,4,0.0776,...,0.000000,5JjnoGJyOxfSZUZtk2rRwZ,1,1,274192,0.01070,6AI3ezQ4o3HUoP6Dhudph3,gangster,audio_features,6mODZNt8CYYEBrWAyS5vs4
182,0.552,144.941,0.640,hip-hop,-5.679,0.119,0.1120,"future, metro boomin, travis scott, playboi carti",4,0.1290,...,0.000000,4iqbFIdGOTzXeDtt9owjQn,1,2,228267,0.02150,28drn6tQo95MRvO0jQEo5C,gangster,audio_features,6mODZNt8CYYEBrWAyS5vs4
220,0.434,150.062,0.833,hip-hop,-8.795,0.165,0.2810,future,4,0.4310,...,0.021900,17FBoXK1NU2rvJBbzdzw0r,1,2,204600,0.01020,0VgkVdmE4gld66l8iyGjgx,trap,audio_features,37i9dQZF1EIhshGKK0SEkb
226,0.413,131.964,0.772,hip-hop,-8.194,0.111,0.2110,chief keef,4,0.0891,...,0.000000,2B4y3j02ho6XNF8BEzx3JF,0,0,246213,0.03610,01Lr5YepbgjXAWR9iOEyH1,trap,audio_features,37i9dQZF1EIhshGKK0SEkb
248,0.583,116.624,0.709,hip-hop,-5.298,0.195,0.5330,"metro boomin, future, chris brown",4,0.2040,...,0.000003,7txGsnDSqVMoRl6RQ9XyZP,0,5,182667,0.13800,0vjeOZ3Ft5jvAi9SBFJm1j,trap,audio_features,37i9dQZF1EIhshGKK0SEkb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0.507,130.007,0.872,hip-hop,-6.323,0.111,0.5060,gunna,4,0.0720,...,0.000000,5HoNV9xqGbGufrIFH271dR,0,4,158667,0.19500,2EKL77lwxvo615Ga7VjEUl,melodic,audio_features,37i9dQZF1EIgLJJI0PCAq9
1599,0.661,76.710,0.836,reggae,-6.502,0.449,0.7530,damian marley,4,0.3380,...,0.000000,4Y0PrDckfFKxKaVXsscDLB,0,7,213200,0.02080,22AbXxQbMdVqEz7xJjhccG,fusion,audio_features,37i9dQZF1EIeAKlEStQOxi
1601,0.787,119.996,0.704,electronic,-6.751,0.159,0.3100,"hugel, topic, arash, daecolm",4,0.0462,...,0.005380,5MrdEMJrBnauBJsd1nYY00,0,11,214000,0.00926,5nPbKG04fhLkIAjcPFaZq7,afro house,audio_features,487jKTFqWhs6b0AEUz0WpX
1609,0.637,120.033,0.847,electronic,-8.751,0.121,0.0509,"zerb, sofiya nzau",4,0.0587,...,0.201000,5asqLE8StjaHpMWSyjzo2D,0,7,208135,0.20100,5KTZgG84bKFGm53lhLtTqc,afro house,audio_features,487jKTFqWhs6b0AEUz0WpX


As observed in the output above, a song (identified by id) can indeed appear multiple times if it's associated with different playlist_genre entries (e.g., a song appearing in a 'pop' playlist and a 'gaming' playlist). This highlights the "Playlist-Based Genre" limitation discussed in the data collection section. 

This will be tackled later on in the notebook ____________________


### Addressing _____ Outliers

The identified "outliers" are songs with __________

In [15]:
from scipy.stats import zscore

# Calculate z-scores for track_popularity
high_df['zscore_popularity'] = zscore(high_df['track_popularity'])

# Identify outliers (z-score > 3 or < -3)
outliers = high_df[(high_df['zscore_popularity'] > 3) | (high_df['zscore_popularity'] < -3)]


print(outliers)

     energy    tempo  danceability playlist_genre  loudness  liveness  \
0     0.592  157.969         0.521            pop    -7.777    0.1220   
1     0.507  104.978         0.747            pop   -10.171    0.1170   
4     0.783  149.027         0.777            pop    -4.477    0.3550   
5     0.582  116.712         0.700            pop    -5.960    0.0881   
455   0.592  157.969         0.521         gaming    -7.777    0.1220   
456   0.507  104.978         0.747         gaming   -10.171    0.1170   
457   0.582  116.712         0.700         gaming    -5.960    0.0881   
676   0.592  157.969         0.521            pop    -7.777    0.1220   
677   0.783  149.027         0.777            pop    -4.477    0.3550   
678   0.507  104.978         0.747            pop   -10.171    0.1170   
688   0.582  116.712         0.700            pop    -5.960    0.0881   

     valence           track_artist  time_signature  speechiness  ...  \
0      0.535  lady gaga, bruno mars               

### Expanding Collaborating Artists
To ensure all contributing artists are accounted for in our analysis of musical evolution, we separate artists listed in `track_artist` (which may contain multiple artists separated by ', ') into individual entries. This allows us to attribute each song to all performers and facilitates artist-level aggregation.

In [16]:
high_df['track_artist'] = high_df['track_artist'].str.split(', ')
high_df_cleaned = high_df.explode('track_artist')
high_df_cleaned = high_df_cleaned.reset_index(drop=True)

We can see how collaboration tracks are now correctly attributed to each individual artist.

Below is an example with Bruno Mars' Die with a Smile, which is a collaboration with Lady Gaga.

In [17]:
high_df_cleaned[high_df_cleaned['track_name'] == 'Die With A Smile'][['track_name', 'track_artist']]



,track_name,track_artist
0,Die With A Smile,lady gaga
1,Die With A Smile,bruno mars
725,Die With A Smile,lady gaga
726,Die With A Smile,bruno mars
1047,Die With A Smile,lady gaga
1048,Die With A Smile,bruno mars


### Assigning artists to genres
As noted earlier, songs can appear in playlists associated with different genres. To analyze an artist's evolution within a specific genre context for our primary research question, we need to assign a single 'primary genre' to each artist. We achieve this by identifying the `playlist_genre` that is most frequently associated with an artist's songs in our dataset. This approach provides a robust representation of an artist's predominant genre.

In [18]:
# Group by artist and count genre occurrences for each artist
artist_genre_counts = high_df_cleaned.groupby(['track_artist', 'playlist_genre']).size().reset_index(name='count')

# Find the most frequent genre for each artist
primary_artist_genres = artist_genre_counts.loc[artist_genre_counts.groupby('track_artist')['count'].idxmax()]
primary_artist_genres = primary_artist_genres[['track_artist', 'playlist_genre']].rename(columns={'playlist_genre': 'primary_genre'})

# Merge this back to the cleaned DataFrame
high_df_cleaned = pd.merge(high_df_cleaned, primary_artist_genres, on='track_artist', how='left')

# Display a few examples to verify
print("Example of artist with their assigned primary genre:")
print(high_df_cleaned[['track_artist', 'primary_genre', 'playlist_genre']].head())

Example of artist with their assigned primary genre:
        track_artist primary_genre playlist_genre
0          lady gaga           pop            pop
1         bruno mars           pop            pop
2      billie eilish           pop            pop
3      gracie abrams           pop            pop
4  sabrina carpenter           pop            pop


### Type Conversions and Feature Definition
To facilitate numerical analysis and proper handling of categorical variables, we perform several type conversions. The track_album_release_date is converted to datetime objects to extract year, month, and day, which are crucial for analyzing temporal evolution. key and mode are converted to categorical types as they represent discrete musical attributes.

We also define lists of numerical and categorical features, excluding identifiers and URLs that are not relevant for feature analysis.

In [19]:
high_df_cleaned['track_album_release_date'] = pd.to_datetime(high_df_cleaned['track_album_release_date'], errors='coerce')
high_df_cleaned['Release Year'] = high_df_cleaned['track_album_release_date'].dt.year
high_df_cleaned['Release Month'] = high_df_cleaned['track_album_release_date'].dt.month
high_df_cleaned['Release Day'] = high_df_cleaned['track_album_release_date'].dt.day

df_cleaned = high_df_cleaned # Renaming for consistency with previous steps, though high_df_cleaned is primary
df_cleaned['key'] = df_cleaned['key'].astype('category')
df_cleaned['mode'] = df_cleaned['mode'].astype('category')

numerical_features = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_cleaned.select_dtypes(include=['object', 'category']).columns.tolist()

non_feature_columns = ['id', 'track_href', 'analysis_url', 'uri', 'type', 'track_album_release_date']

# Define features that will be used for analysis. We exclude non-feature columns
features = df_cleaned.columns.drop(non_feature_columns).tolist()

print("Initial numerical features identified:", numerical_features)
print("Initial categorical features identified:", categorical_features)

# maybe also scale numerical features if needed
# from sklearn.preprocessing import StandardScaler

Initial numerical features identified: ['energy', 'tempo', 'danceability', 'loudness', 'liveness', 'valence', 'time_signature', 'speechiness', 'track_popularity', 'instrumentalness', 'duration_ms', 'acousticness', 'zscore_popularity', 'Release Year', 'Release Month', 'Release Day']
Initial categorical features identified: ['playlist_genre', 'track_artist', 'track_href', 'uri', 'track_album_name', 'playlist_name', 'analysis_url', 'track_id', 'track_name', 'track_album_id', 'mode', 'key', 'id', 'playlist_subgenre', 'type', 'playlist_id', 'primary_genre']


### Quantifying Musical Change
TODO: complete this

How can we mathematically define and measure an artist's "musical change" over time using the available audio features?

Before we can compare artists or genres, we must first establish a **consistent and quantifiable** metric for `musical change` using the available audio features. In essence, this question forces us to define our dependent variable.

In [ ]:
# List of musical features to track
features_to_track = [
    'danceability', 'energy', 'acousticness',
    'valence', 'tempo', 'instrumentalness'
]

# Group by artist and compute standard deviation for each feature
artist_feature_std = df_cleaned.groupby('track_artist')[features_to_track].std()

# Fill NaN values with 0 for artists with no variation in features
artist_feature_std = artist_feature_std.fillna(0)

# Calculate a single "musical change score" by averaging the std values
artist_feature_std['musical_change_score'] = artist_feature_std.mean(axis=1)

artist_feature_std


,danceability,energy,acousticness,valence,tempo,instrumentalness,musical_change_score
track_artist,,,,,,,
*nsync,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
.diedlonely,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
21 savage,0.133401,0.140226,0.167380,0.159925,36.971280,2.603826e-02,6.266375
24kgoldn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
2pac,0.130185,0.078460,0.147924,0.186268,40.802599,2.765508e-03,6.891367
...,...,...,...,...,...,...,...
¥$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
énouement,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
ñengo flow,0.164545,0.096417,0.091625,0.045033,1.267861,7.159143e-07,0.277581


## Exploratory Data Analysis

To help us answer the research question, we have chosen the following EDA questions:

### 1. MISSING - make sure its got a visual aspect so maybe like see a distribution or something !!!!!!!!!!!

### 2. What is the distribution of artists across the dataset's `Playlist Genres`?
This is to understand the composition of our dataset and check for **feasibility**. We need to ensure there are enough artists in each genre to make meaningful comparisons. If one genre is heavily overrepresented or another is barely present, it could affect the validity of our conclusions.

### 3. For a few sample artists from different genres, what do their key audio features look like when plotted against their album release dates? 
This is to visually validate our core premise. By plotting the trajectories of individual artists, we can get a first look on whether musical change is observable in the data. (just to make sure were not wasting our time lol)
